In [1]:
import numpy as np
import pandas as pd
import math as m
import sympy as sym
import scipy as sci
import matplotlib.ticker as ticker
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
from IPython.display import display, Latex
from sympy import pprint


In [2]:
from scipy.optimize import minimize as GDlib

In [3]:
# Prev version - until March 08
# def Creating_states( c = [0, m.pi/2] , priors = [.5,.5], input_type = 'theta'):    # coeff list like [a0,a1,b0,b1]
    
#     if input_type == "coeff":
#         if len(c) != 4:
#             print("Error! Coefficients given in raw form have to be in form [a0,a1,b0,b1]")
#         coeff = c
    
#     if input_type == "theta":
#         if len(c) != 2:
#             print("Error! Coefficients given in theta form have to be angles: [theta_0, delta_theta]")
    
#     theta_0 , theta = c[0], c[1]
#     coeff = [np.cos(theta_0), np.sin(theta_0), np.cos(theta_0+theta), np.sin(theta_0+theta)] # the first state here is the theta_0 angle 
#     coeff = [np.round(i,9) for i in coeff]                                                   ## from |0> axis, the second theta_0 + theta
    
        
#     a0, a1, b0, b1 = coeff[0],coeff[1],coeff[2],coeff[3]
#     psi0, psi1 = [a0,a1], [b0,b1]      # defining states
    
#     psi0sq, psi1sq = [], []
#     [[psi0sq.append(i*j) for i in psi0] for j in psi0]
#     [[psi1sq.append(i*j) for i in psi1] for j in psi1]   
    
#     psi0psi0, psi1psi1 = [psi0sq[0], (np.sqrt(2))*psi0sq[1], psi0sq[3]], [psi1sq[0], (np.sqrt(2))*psi1sq[1], psi1sq[3]] # squared
#     vec_psi0psi0, vec_psi1psi1 = np.array(psi0psi0), np.array(psi1psi1)     # np array conversion 
#     rho = priors[0]*np.outer(vec_psi0psi0, vec_psi0psi0)+ priors[1]*np.outer(vec_psi1psi1, vec_psi1psi1)     # theoretical rho
    
#     return([psi0,psi1], rho)

# Creating_states([0,m.pi/4])[1]



In [4]:
#version on Mar 12
# # MLE stuff
# theta1, theta2, phi1, phi2, alpha = sym.symbols('theta1 theta2 phi1 phi2 alpha')

# def Creating_states( theta = [0, m.pi/2], phi = [0,0] , priors = [.5,.5], input_type = 'theta', Abstract = False):    # coeff list like [a0,a1,b0,b1]
#     # if Abstract == True:
#     #     theta1, theta2 = sym.symbols('theta1 theta2')
#     #     a0, a1 = sym.cos(theta1), sym.exp(sym.I*phi1)*sym.sin(theta1)
#     #     b0, b1 = sym.cos(theta2), sym.exp(sym.I*phi2)*sym.sin(theta2)
#     # theta1, theta2 = sym.symbols('theta1 theta2')
#     a0, a1 = sym.cos(theta1), sym.exp(sym.I*phi1)*sym.sin(theta1)
#     b0, b1 = sym.cos(theta2), sym.exp(sym.I*phi2)*sym.sin(theta2)
#     # else:
#     #     theta1 , theta2 = c[0], c[1]
#     #     a0, a1 = sym.cos(theta1), sym.sin(theta1)
#     #     b0, b1 = sym.cos(theta2), sym.sin(theta2)
    
#     alpha = sym.symbols('alpha')
#     if Abstract == False:
#         a0, a1, b0, b1 = a0.subs({theta1 : theta[0]}), a1.subs({theta1 : theta[0], phi1 : phi[0]}), b0.subs({theta2 : theta[1]}), b1.subs({theta2 : theta[1], phi2: phi[1]})
#         alpha = np.arccos(np.sqrt(priors[0])) 
    
#     psi0, psi1 = [a0,a1], [b0,b1]      # defining states

#     psi0psi0_list, psi1psi1_list = [psi0[0]*psi0[0], (np.sqrt(2)*psi0[0]*psi0[1]) , psi0[1]*psi0[1]],  [psi1[0]*psi1[0], (np.sqrt(2)*psi1[0]*psi1[1]), psi1[1]*psi1[1]] # @@@ the square rooting is simplistic. see notebook for better
#     psi0psi0, psi1psi1 = sym.Matrix(psi0psi0_list), sym.Matrix(psi1psi1_list)
#     rho = sym.cos(alpha)**2*(psi0psi0*psi0psi0.T)+ sym.sin(alpha)**2*(psi1psi1*psi1psi1.T)

#     return([[psi0,psi1], [psi0psi0,psi1psi1], rho])
# x = Creating_states(theta = [0,m.pi/2], Abstract=0)[0]
# y = Creating_states(theta = [0,m.pi/2], Abstract=0)[1]
# z = Creating_states(theta = [0,m.pi/2], Abstract=0)[2]
# # tt = y[1].subs({theta1 : 0, theta2 : -m.pi/4, alpha: m.pi/4})
# # tt
# # z.subs({theta1 : 0, theta2 : -m.pi/4, alpha: m.pi/4, phi1: 0, phi2: 0})
# z

In [5]:
# MLE stuff
theta1, theta2, phi1, phi2, alpha = sym.symbols('theta1 theta2 phi1 phi2 alpha')

def Creating_states( params = [0, m.pi/2,0, 0, m.pi/4], input_type = 'theta', Abstract = False):    # coeff list like [a0,a1,b0,b1]
    # if Abstract == True:
    #     theta1, theta2 = sym.symbols('theta1 theta2')
    #     a0, a1 = sym.cos(theta1), sym.exp(sym.I*phi1)*sym.sin(theta1)
    #     b0, b1 = sym.cos(theta2), sym.exp(sym.I*phi2)*sym.sin(theta2)
    # theta1, theta2 = sym.symbols('theta1 theta2')
    a0, a1 = sym.cos(theta1), sym.exp(sym.I*phi1)*sym.sin(theta1)
    b0, b1 = sym.cos(theta2), sym.exp(sym.I*phi2)*sym.sin(theta2)
    # else:
    #     theta1 , theta2 = c[0], c[1]
    #     a0, a1 = sym.cos(theta1), sym.sin(theta1)
    #     b0, b1 = sym.cos(theta2), sym.sin(theta2)
    
    alpha = sym.symbols('alpha')
    if Abstract == False:
        alpha = params[4] 
        a0, a1, b0, b1 = a0.subs({theta1 : params[0]}), a1.subs({theta1 : params[0], phi1 : params[2]}), b0.subs({theta2 : params[1]}), b1.subs({theta2 : params[1], phi2: params[3]})
        alpha = params[4] 
        coeff_raw = [a0,a1, b0, b1]
        coeff = [i if abs(i)> 1e-9 else 0 for i in coeff_raw]
        a0, a1, b0, b1 = coeff[0],coeff[1],coeff[2],coeff[3]
        # print(coeff)
    
    psi0, psi1 = sym.Matrix([a0,a1]), sym.Matrix([b0,b1])      # defining states
    
    psi0psi0_list, psi1psi1_list = [psi0[0]*psi0[0], (np.sqrt(2)*psi0[0]*psi0[1]) , psi0[1]*psi0[1]],  [psi1[0]*psi1[0], (np.sqrt(2)*psi1[0]*psi1[1]), psi1[1]*psi1[1]] # @@@ the square rooting is simplistic. see notebook for better
    psi0psi0, psi1psi1 = sym.Matrix(psi0psi0_list), sym.Matrix(psi1psi1_list)
    rho = sym.cos(alpha)**2*(psi0psi0*psi0psi0.T)+ sym.sin(alpha)**2*(psi1psi1*psi1psi1.T)
    return([[psi0,psi1], [psi0psi0,psi1psi1], rho])

x = Creating_states(params = [0,m.pi/2, 0, 0, m.pi/4], Abstract=0)[0]
y = Creating_states(params = [0,m.pi/2, 0, 0, m.pi/4], Abstract=0)[1]
z = Creating_states(params = [0,m.pi/2, 0, 0, m.pi/4], Abstract=1)[2]
z1 = Creating_states(params = [0,m.pi/4, 0, 0, m.pi/4], Abstract=0)[2]
# tt = y[1].subs({theta1 : 0, theta2 : -m.pi/4, alpha: m.pi/4})
# tt
# z.subs({theta1 : 2, theta2 : -m.pi/4, alpha: m.pi/4, phi1: 0, phi2: 0}).evalf()
sym.pprint(z1) # using the function
z.subs({theta1 : 0, theta2 : m.pi/2, alpha: m.pi/4, phi1: 0, phi2: 0}).evalf()      # manually subbing
# z




⎡      0.625        0.176776695296637        0.125      ⎤
⎢                                                       ⎥
⎢0.176776695296637        0.25         0.176776695296637⎥
⎢                                                       ⎥
⎣      0.125        0.176776695296637        0.125      ⎦


Matrix([
[                 0.5, 1.62340758336808e-49, 1.87469972832732e-33],
[1.62340758336808e-49, 3.74939945665464e-33, 4.32978028117747e-17],
[1.87469972832732e-33, 4.32978028117747e-17,                  0.5]])

In [6]:
#rough
a1, a2 = 1,1
b1,b2 = 2+sym.I, 3+2*sym.I

qubit1 = sym.Matrix([a1, b1])
qubit2 = sym.Matrix([a2, b2])

# Compute the inner product, taking the complex conjugate of the first qubit
inner_product = qubit1.H * qubit2
inner_product2 = qubit2*qubit1.H

# print(type(inner_product.evalf()))
# sym.pprint(inner_product.evalf())

qubit1[1]

2 + I

In [7]:
w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # an array of POVM direction vectors
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrices

In [8]:
#rough


In [9]:
def experiment(N = 10000, params = [0,m.pi/2,0,0, m.pi/2], show_calcs = False, input_type = 'theta'):
    rho = Creating_states( params = params, Abstract = 0)[-1] # theoretical rho
    #Th. prob
    prob_vec_sympy =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)]    # created list of Th probabilities
    prob_vec_raw = [(float(i.as_real_imag()[0])+float(i.as_real_imag()[1])*1j) for i in prob_vec_sympy]  # this is to avoid error, to convert sympy float to ordinary number
    prob_vec = [round(i.real, 10) for i in prob_vec_raw if abs(i.imag) < .0001]          # cleaned up theoretical prob vector

    #collapse
    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials
    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]
    pj_num_vec = [i/N for i in nj_vec]                                  # numerical prob vector     
    return [prob_vec, nj_vec]

print("calculated probs",experiment()[1])

# [[0.0833333333,   0.1666666667,   0.0833333333,   0.0833333333,   0.1666666667,   0.0833333333,   0.0833333333,   0.1666666667,   0.0833333333],
#  [850, 1623, 839, 850, 1665, 879, 784, 1719, 791]]

calculated probs [1684, 1713, 0, 1636, 1640, 0, 1671, 1656, 0]


In [10]:
alpha, phi1, phi2, theta1, theta2 = sym.symbols('alpha phi1 phi2 theta1 theta2')
w1, w2 = sym.cos(alpha)**2, sym.sin(alpha)**2
c1, c2, s1, s2 = sym.cos(theta1), sym.cos(theta2), sym.sin(theta1), sym.sin(theta2)

# # analytic formuale for the probabilities, as given in the notes
# probv1 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1)*s1*c1)+w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2)*s2*c2))
# probv2 = (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1)*c1*s1) + w2*c2**2*(1+s2**2*-2*m.sqrt(2)*sym.cos(phi2)*c2*s2))
# probv3 = (1/6)*(w1*(c1**4+s1**4 -2*sym.cos(2*phi1)*s1**2*c1**2)+ w2*(c2**4+ s2**4 - 2*sym.cos(2*phi2)*s2**2*c2**2))
# probv4 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1+2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2+2*m.pi/3)*s2*c2))
# probv5 = (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
# probv6 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1 - 2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2 - 2*m.pi/3)*c2**2*s2**2))
# probv7 = (1/6)*(w1*s1**2*(1+c1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*s2*c2))
# probv8 = (1/6)*(w1*c1**2*(1+s1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
# probv9 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1+2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2+2*m.pi/3)*c2**2*s2**2))
## prob = [pr1, pr2,pr3, pr4,pr5, pr6,pr7, pr8,pr9]

# But the order is taken different from what we took in the numerical part (see MLE notes)...so rearranging the labels:

pr1 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1)*s1*c1)+w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2)*s2*c2))
pr2 = (1/6)*(w1*(c1**4+s1**4 -2*sym.cos(2*phi1)*s1**2*c1**2)+ w2*(c2**4+ s2**4 - 2*sym.cos(2*phi2)*s2**2*c2**2))
pr3 = (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1)*c1*s1) + w2*c2**2*(1+s2**2*-2*m.sqrt(2)*sym.cos(phi2)*c2*s2))
pr4 = (1/6)*(w1*s1**2*(1+c1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*s2*c2))
pr5 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1+2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2+2*m.pi/3)*c2**2*s2**2))
pr6 = (1/6)*(w1*c1**2*(1+s1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
pr7 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1+2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2+2*m.pi/3)*s2*c2))
pr8 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1 - 2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2 - 2*m.pi/3)*c2**2*s2**2))
pr9 = (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
pr = [pr1, pr2,pr3, pr4,pr5, pr6,pr7, pr8,pr9]   # now the probabilites should match the numerical one


In [ ]:
"""# Likelihood fn

pr_analytical = [i.subs({theta1 : 0, theta2 : m.pi/2, phi1 : 0, phi2 : 0, alpha : m.pi/4}) for i in pr]
print("analytical:",pr_analytical)
# print(sum(pr_analytical))

#SEMI ANALYTICAL
rho = Creating_states(Abstract=1)[-1]
pr_semanl_abs = [np.trace(np.dot(rho, POVM_elts[i])) for i in range(len(POVM_elts))]
pr_semanl = [i.subs({theta1 : 0, theta2 : m.pi/2, phi1 : 0, phi2 : 0, alpha : m.pi/4}) for i in pr_semanl_abs]
print("semi anlt",pr_semanl) 


# Numerical
rho = Creating_states(Abstract=0)[-1]
pr_num = [np.trace(np.dot(rho, POVM_elts[i])) for i in range(len(POVM_elts))]
print("mun",pr_num) 


# numerical from the experimaent function
pr_exp = experiment(N = 100000, theta = [0,m.pi/2])[0]
print("exp",pr_exp) 

#corresponding n vector
n = experiment(N = 100000, theta = [0,m.pi/2])[1]
print("n vector:",n) 
# verified, that all methods Converge to the same probabilities."""

'# Likelihood fn\n\npr_analytical = [i.subs({theta1 : 0, theta2 : m.pi/2, phi1 : 0, phi2 : 0, alpha : m.pi/4}) for i in pr]\nprint("analytical:",pr_analytical)\n# print(sum(pr_analytical))\n\n#SEMI ANALYTICAL\nrho = Creating_states(Abstract=1)[-1]\npr_semanl_abs = [np.trace(np.dot(rho, POVM_elts[i])) for i in range(len(POVM_elts))]\npr_semanl = [i.subs({theta1 : 0, theta2 : m.pi/2, phi1 : 0, phi2 : 0, alpha : m.pi/4}) for i in pr_semanl_abs]\nprint("semi anlt",pr_semanl) \n\n\n# Numerical\nrho = Creating_states(Abstract=0)[-1]\npr_num = [np.trace(np.dot(rho, POVM_elts[i])) for i in range(len(POVM_elts))]\nprint("mun",pr_num) \n\n\n# numerical from the experimaent function\npr_exp = experiment(N = 100000, theta = [0,m.pi/2])[0]\nprint("exp",pr_exp) \n\n#corresponding n vector\nn = experiment(N = 100000, theta = [0,m.pi/2])[1]\nprint("n vector:",n) \n# verified, that all methods Converge to the same probabilities.'

240934.192083837

In [ ]:
#rough
print(lnL_list[0])
print(lnL_list[0].subs({phi1:0, phi2:0, theta1: 0, theta2: m.pi/2,  alpha: m.pi/4}))
print(lnL_list[1].subs({phi1:0, phi2:0, theta1: 0, theta2: m.pi/2,  alpha: m.pi/4}))
print(lnL_list[2].subs({phi1:0, phi2:0, theta1: 0, theta2: m.pi/2,  alpha: m.pi/4}))
print(lnL_list[3].subs({phi1:0, phi2:0, theta1: 0, theta2: m.pi/2,  alpha: m.pi/4}))
print(lnL_list[4].subs({phi1:0, phi2:0, theta1: 0, theta2: m.pi/2,  alpha: m.pi/4}))
print(lnL_list[5].subs({phi1:0, phi2:0, theta1: 0, theta2: m.pi/2,  alpha: m.pi/4}))
print(lnL_list[6].subs({phi1:0, phi2:0, theta1: 0, theta2: m.pi/2,  alpha: m.pi/4}))
print(lnL_list[7].subs({phi1:0, phi2:0, theta1: 0, theta2: m.pi/2,  alpha: m.pi/4}))
print(lnL_list[8].subs({phi1:0, phi2:0, theta1: 0, theta2: m.pi/2,  alpha: m.pi/4}))
k = [i.subs({phi1:0, phi2:0, theta1: 0, theta2: m.pi/2, alpha: m.pi/4}) for i in lnL_list]
print(k)
print('sum', sum(k))

8305*log(0.166666666666667*(-2.82842712474619*sin(theta1)*cos(phi1)*cos(theta1) + cos(theta1)**2 + 1)*sin(theta1)**2*cos(alpha)**2 + 0.166666666666667*(-2.82842712474619*sin(theta2)*cos(phi2)*cos(theta2) + cos(theta2)**2 + 1)*sin(alpha)**2*sin(theta2)**2)
-20637.1497264893
-30062.1403747083
-21004.9159106580
-20699.2723927340
-30175.0212212697
-20460.7213543544
-20582.4817801940
-29743.2071891857
-20316.5967686667
[-20637.1497264893, -30062.1403747083, -21004.9159106580, -20699.2723927340, -30175.0212212697, -20460.7213543544, -20582.4817801940, -29743.2071891857, -20316.5967686667]
sum -213681.506718260


In [ ]:
#manual checking, rough
v = [(lnL.subs({theta1 : 0, theta2 : i, phi1 : 0, phi2 : 0, alpha : m.pi/4}), i*(180/m.pi)) for i in np.linspace(-m.pi/12,m.pi+m.pi/2+m.pi/12, 300)]
v
# works for theta 2 variable; gives two poits at which the lnL is minimum : 90 and 270. Further searching in phi should resolve the ambiguity.

[(-254552.186543513, -14.999999999999998),
 (-257130.016520079, -13.996655518394647),
 (-260000.391754356, -12.993311036789297),
 (-263206.319296245, -11.989966555183946),
 (-266801.622963067, -10.986622073578594),
 (-270855.110142418, -9.983277591973243),
 (-275456.951485867, -8.979933110367892),
 (-280728.855492690, -7.976588628762542),
 (-286841.119074501, -6.97324414715719),
 (-294043.015750206, -5.969899665551838),
 (-302721.404709337, -4.966555183946488),
 (-313526.343468499, -3.963210702341138),
 (-327683.694396780, -2.959866220735786),
 (-347976.990248950, -1.956521739130434),
 (-383531.565176199, -0.9531772575250836),
 (-530042.210683179, 0.05016722408026664),
 (-378143.346212534, 1.0535117056856185),
 (-344686.013025985, 2.0568561872909705),
 (-324829.998923023, 3.0602006688963224),
 (-310701.103352375, 4.063545150501671),
 (-299767.402842458, 5.0668896321070225),
 (-290888.310695157, 6.070234113712375),
 (-283451.511427088, 7.0735785953177235),
 (-277090.032610238, 8.0769230

In [ ]:
#defining gradient function
def grad( params=[0,m.pi/2,0,0,m.pi/4] ,s=(1/500), show_calcs = False):    # point is the list of parameter : [theta1, theta2, phi1, phi2, alpha] respectively
    
    del_theta1 = logL(params = [params[0]+s,params[1],params[2],params[3],params[4]]) - logL(params = [params[0],params[1],params[2],params[3],params[4]])
    del_theta2 = logL(params = [params[0],params[1]+s,params[2],params[3],params[4]]) - logL(params = [params[0],params[1],params[2],params[3],params[4]])
    del_phi1   = logL(params = [params[0],params[1],params[2]+s,params[3],params[4]]) -   logL(params = [params[0],params[1],params[2],params[3],params[4]])
    del_phi2   = logL(params = [params[0],params[1],params[2],params[3]+s,params[4]]) -   logL(params = [params[0],params[1],params[2],params[3],params[4]])
    del_alpha  = logL(params = [params[0],params[1],params[2],params[3],params[4]+s]) -  logL(params = [params[0],params[1],params[2],params[3],params[4]])

    grad = [del_theta1/s,del_theta2/s,del_phi1/s,del_phi2/s,del_alpha/s]
    grad = [i if abs(i) > 1e-5 else 0 for i in grad]
    # for i in grad:
    #     if i > 1e9 or i is False:
    #         print(f"something wrong, gradient blowing up for parameter {i}")
    return grad
print(grad())


[802.963943933719, 121.725266246358, 0, 0, 89.1242376819719]


In [ ]:
# rough
s = 1/200

a = [-3.739e-01, 1.595e+00, 1.346e+00, -1.553e+00, 7.458e-01]
b = [0, m.pi/2, 0, 0, m.pi/4]
v0 = "original",logL(a)
v1 = "new",logL(b)

print("original",logL(a))
print("new",logL(b))

print("grad",grad([-0.4, .195, .346, -.553, 0.7458]))
b

original 205479.837211471
new 213681.506718260
grad [43961.3730165001, -42927.3838519148, -5530.56985090370, 1766.54624816729, 8904.13081909355]


[0, 1.5707963267948966, 0, 0, 0.7853981633974483]

In [ ]:
#Grad descent
params_manual = [0,1,0,0,m.pi/4] # same 
def GD(param0 = [0,m.pi/2,0,0,m.pi/4], nit = 30, step = 1/200, show_calc = False):
    cost0 = logL(params0)
    s = step
    grad_new = grad(params = params0)
    params_new = params0
    values = []

    for i in range(nit):
        params_new = [params_new[i]-s*(grad_new[i]/8000) for i in range(len(params0))]
        grad_new = grad(params_new)
        value = [logL(params_new), params_new, grad_new, i]
        values.append(value)
        
        if i > (3/10)*nit//1 and i > 10:
            s = 5*step
            if i > (8/10)*nit//1:
                s = 15*step


        if show_calc == True:
            print(value,'\n ')
    return [params_new]

# GD(param0= params_manual, nit= 150, step = 1/200, show_calc = True)

[222348.251246988, [-0.00119311254594322, 1.24365144717464, 0.0, -0.000824420218532396, 0.782998137276171], [4618.59045790334, -62842.0725960314, -38.7847042293288, 990.672311818344, -2971.28002575482], 0] 
 
[220081.922334128, [-0.00407973158213281, 1.28292774254716, 2.42404401433305e-5, -0.00144359041341886, 0.784855187292268], [3677.18061339110, -51342.5351533369, -139.003601099830, 741.123423926183, -2724.16744062502], 1] 
 
[218548.844133760, [-0.00637796946550225, 1.31501682701800, 0.000111117690830724, -0.00190679255337272, 0.786557791942659], [2859.98128706706, -43080.6675834901, -225.299475758220, 557.888458497473, -2274.23597090819], 2] 
 
[217460.743574703, [-0.00816545776991916, 1.34194224425768, 0.000251929863179612, -0.00225547283993365, 0.787979189424476], [2189.10398434673, -36794.2462494393, -296.708956811926, 422.450259764446, -1773.36065382406], 3] 
 
[216662.780529303, [-0.00953364776013586, 1.36493864816358, 0.000437372961187066, -0.00251950425228642, 0.78908753983

[[-0.372512890911648,
  1.56155485349067,
  1.33549196427934,
  0.00701260972516138,
  0.746228887535807]]

In [ ]:
# [0, 1.5707963267948966, 0, 0, 0.7853981633974483]
#  [-0.0453554437925232, 1.56970457797804, 0.0969275280091279, -0.00179414849833847, 0.784096793392809]
#  [-0.372572520872864, 1.55890221385790, 1.23967731543922, 0.00149623378345495, 0.746416071840694]


42.742651516759416

In [19]:
# n =[8305, 16778, 8453, 8330, 16841, 8234, 8283, 16600, 8176]    # for statres [0,m.pi/2]
n = [1678, 0, 1639, 1656, 0, 1662, 1723, 0, 1642]
n_normalized =[i/sum(n) for i in n]     # "experimental probabililtes"

#defining likelihood expression
lnL_list = [n[i]*sym.log(pr[i]) for i in range(len(pr))]
lnL = sum(lnL_list)         # created the likelihood fucntion
lnL_psuedo = sum([n[i]*m.log(n_normalized[i]) for i in range(len(n))])

# print("ideal answer",lnL.subs({phi1:0, phi2:0, theta1: 0, alpha: m.pi/4, theta2: m.pi/2})) # ideal answer -213681.506718260
# print("psuedo answer",lnL_psuedo)         # psuedo answer -213676.9481968622

#Defining likelihood as a funciton of parameters.
def logL(params = [0,m.pi/2,0,0,m.pi/4]):
    global lnL
    logL = lnL.subs({theta1: params[0], theta2: params[1], phi1: params[2] , phi2: params[3] , alpha: params[4]})
    
    if logL < -500000:
        print(logL, params)

    return (-logL.evalf())
    

params0 = [0,1,0,0,m.pi/4]
# bounds = [(-1,1), (-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180)]
bounds = [(-m.pi,m.pi+m.pi/6), (-m.pi,m.pi+m.pi/6),(-m.pi,m.pi+m.pi/6),(-m.pi,m.pi+m.pi/6), (0, m.pi/2 + m.pi/20)]


print('ideal = ' , lnL_psuedo)

result = GDlib(logL, params0, bounds= bounds)
result

ValueError: math domain error

In [ ]:
s = Creating_states(params = result.x)[0]
t = Creating_states()[0]

pprint(s)
pprint(t)
# sym.pprint(s)

fid1 = abs(sym.Matrix(s[0]).H*t[0])**2
fid2 = abs(sym.Matrix(s[1]).H*t[1])**2
fid = [fid1, fid2]
pprint(fid)

⎡⎡          0.930907673429997           ⎤  ⎡         -0.0239573001262181      
⎢⎢                                      ⎥  ⎢                                  
⎢⎢                    1.34611426055477⋅ⅈ⎥, ⎢                   -1.553343034274
⎣⎣-0.365254573618387⋅ℯ                  ⎦  ⎣0.999712982695865⋅ℯ               

    ⎤⎤
    ⎥⎥
95⋅ⅈ⎥⎥
    ⎦⎦
⎡⎡1⎤  ⎡ 0 ⎤⎤
⎢⎢ ⎥, ⎢   ⎥⎥
⎣⎣0⎦  ⎣1.0⎦⎦
[[0.86658909645085], [0.999426047770662]]


In [ ]:
def GDSearch(params0 = [0,1,0,0,m.pi/4], bounds = [(-1,1), (-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180)]):
    result = GDlib(logL, params0, bounds= bounds)
    params = result.x
    vecs = Creating_states(params = params, Abstract = 0)[0]
    if result.success is not True:
        print("\n CHECK \n- didnt succeed? \n ")
    phi1_num, phi2_num = sym.Matrix(vecs[0]), sym.Matrix(vecs[1])
    return([phi1_num, phi2_num], result)

GDSearch()[0]

[Matrix([
 [                         0.930907673429997],
 [-0.365254573618387*exp(1.34611426055477*I)]]),
 Matrix([
 [                       -0.0239573001262181],
 [0.999712982695865*exp(-1.55334303427495*I)]])]